# Manipulate data (code from orignial source modified for windows and jupyter notebook

In [1]:
import os
import shutil
import multiprocessing
import pandas as pd
from datetime import timedelta, datetime
import numpy as np

stress_data_path = "C:/Users/jatin/OneDrive/Desktop/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset"
cpu_count = int(multiprocessing.cpu_count()/2)

new_list = [
    (file, sub_file) 
    for file in os.listdir(stress_data_path) 
    for sub_file in os.listdir(os.path.join(stress_data_path, file))
]
for i in range(len(new_list)):
    file=new_list[i][0]
    sub_file=new_list[i][1]
    shutil.unpack_archive(
        os.path.join(stress_data_path, file, sub_file), 
        os.path.join(stress_data_path, file, sub_file[:-4])
    )


In [13]:
import os
import shutil
import multiprocessing
import pandas as pd
from datetime import timedelta, datetime
import numpy as np

DATA_PATH = "C:/Users/jatin/OneDrive/Desktop/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/Stress_dataset"
SAVE_PATH = "C:/Users/jatin/OneDrive/Desktop/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/processed_dataset"
os.mkdir(SAVE_PATH)

final_columns = {
    'ACC': ['id', 'X', 'Y', 'Z', 'datetime'],
    'EDA': ['id', 'EDA', 'datetime'],
    'HR': ['id', 'HR', 'datetime'],
    'TEMP': ['id', 'TEMP', 'datetime'],
}

names = {
    'ACC.csv': ['X', 'Y', 'Z'],
    'EDA.csv': ['EDA'],
    'HR.csv': ['HR'],
    'TEMP.csv': ['TEMP'],
}

desired_signals = ['ACC.csv', 'EDA.csv', 'HR.csv', 'TEMP.csv']
#desired_signals = ['ACC.csv', 'EDA.csv', 'HR.csv']

acc = pd.DataFrame(columns=final_columns['ACC'])
eda = pd.DataFrame(columns=final_columns['EDA'])
hr = pd.DataFrame(columns=final_columns['HR'])
temp = pd.DataFrame(columns=final_columns['TEMP'])

def process_df(df, file):
    start_timestamp = df.iloc[0,0]
    sample_rate = df.iloc[1,0]
    new_df = pd.DataFrame(df.iloc[2:].values, columns=df.columns)
    new_df['id'] =  file[-2:]
    new_df['datetime'] = [(start_timestamp + i/sample_rate) for i in range(len(new_df))]
    return new_df
c1=0
c2=0
c3=0
c4=0
for file in os.listdir(DATA_PATH):
    print(f'Processing {file}')
    for sub_file in os.listdir(os.path.join(DATA_PATH, file)):
        if not sub_file.endswith(".zip"):
            for signal in os.listdir(os.path.join(DATA_PATH, file, sub_file)):
                if signal in desired_signals:
                    df = pd.read_csv(os.path.join(DATA_PATH, file, sub_file, signal), names=names[signal], header=None)
                    if not df.empty:
                        if signal == 'ACC.csv':
                            st="acc"+str(c1)+".csv"
                            process_df(df, file).to_csv(os.path.join(SAVE_PATH, st), index=False)
                            c1+=1
                        if signal == 'EDA.csv':
                            st="eda"+str(c2)+".csv"
                            process_df(df, file).to_csv(os.path.join(SAVE_PATH, st), index=False)
                            c2+=1
                        if signal == 'HR.csv':
                            st="hr"+str(c3)+".csv"
                            process_df(df, file).to_csv(os.path.join(SAVE_PATH, st), index=False)
                            c3+=1
                        if signal == 'TEMP.csv':
                            st="temp"+str(c4)+".csv"
                            process_df(df, file).to_csv(os.path.join(SAVE_PATH, st), index=False)
                            c4+=1

print('Done Saving Data ...')



Processing 15
Processing 5C
Processing 6B
Processing 6D
Processing 7A
Processing 7E
Processing 83
Processing 8B
Processing 94
Processing BG
Processing CE
Processing DF
Processing E4
Processing EG
Processing F5
Done Saving Data ...


## merge all files now

In [1]:
import os
import shutil
import multiprocessing
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
from tqdm import tqdm
 


In [4]:
SAVE_PATH = "C:/Users/jatin/OneDrive/Desktop/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/final"
c=0
for i in range(609):
    st="/"+"merged_data"+str(i)+".csv"
    df = pd.read_csv(SAVE_PATH+st, dtype={'id': str})
    c+=df.shape[0]
print(sorted(c,lambda x:x[1]))

143634730


In [2]:

COMBINED_DATA_PATH = "C:/Users/jatin/OneDrive/Desktop/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/processed_dataset"
SAVE_PATH = "C:/Users/jatin/OneDrive/Desktop/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/final"

# if COMBINED_DATA_PATH != SAVE_PATH:
#     os.mkdir(SAVE_PATH)

print("Reading data ...")
for j in tqdm(range(113,609)):
    
    acc, eda, hr, temp = None, None, None, None

    signals = ['acc', 'eda', 'hr', 'temp']
    results=[]
    for i in range(len(signals)):

        signal=signals[i]
        st="/"+str(signal)+str(j)+".csv"
        df = pd.read_csv(COMBINED_DATA_PATH+st, dtype={'id': str})
        results.append([signal,df])
    for i in results:
        globals()[i[0]] = i[1]

    # Merge data
    ids = eda['id'].unique()
    columns=['X', 'Y', 'Z', 'EDA', 'HR', 'TEMP', 'id', 'datetime']
    results1=[]
    for i in range(len(ids)):
        id=ids[i]
        df = pd.DataFrame(columns=columns)

        acc_id = acc[acc['id'] == id]
        eda_id = eda[eda['id'] == id].drop(['id'], axis=1)
        hr_id = hr[hr['id'] == id].drop(['id'], axis=1)
        temp_id = temp[temp['id'] == id].drop(['id'], axis=1)

        df = acc_id.merge(eda_id, on='datetime', how='outer')
        df = df.merge(temp_id, on='datetime', how='outer')
        df = df.merge(hr_id, on='datetime', how='outer')

        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)
        results1.append(df)






    new_df = pd.concat(results1, ignore_index=True)

    st="merged_data"+str(j)+".csv"
    new_df.to_csv(os.path.join(SAVE_PATH, st), index=False)

  0%|                                                                                          | 0/496 [00:00<?, ?it/s]

Reading data ...


100%|████████████████████████████████████████████████████████████████████████████████| 496/496 [22:43<00:00,  2.75s/it]


In [12]:


# Read Files
print("Reading 1 ...")
PATH = 'C:/Users/t07/Downloads/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/majid2'

df = pd.read_csv(os.path.join(PATH, 'merged_data.csv'), dtype={'id': str})
df['datetime'] = pd.to_datetime(df['datetime'].apply(lambda x: x * (10 ** 9)))

print("Reading 2 ...")
survey_path = 'C:/Users/t07/Downloads/jstin/doi_10.5061_dryad.5hqbzkh6f__v6/SurveyResults.xlsx'

survey_df = pd.read_excel(survey_path, usecols=['ID', 'Start time', 'End time', 'date', 'Stress level'], dtype={'ID': str})
survey_df['Stress level'].replace('na', np.nan, inplace=True)
survey_df.dropna(inplace=True)

survey_df['Start datetime'] =  pd.to_datetime(survey_df['date'].map(str) + ' ' + survey_df['Start time'].map(str))
survey_df['End datetime'] =  pd.to_datetime(survey_df['date'].map(str) + ' ' + survey_df['End time'].map(str))
survey_df.drop(['Start time', 'End time', 'date'], axis=1, inplace=True)

# Convert SurveyResults.xlsx to GMT-00:00
print("Converting ...")
daylight = pd.to_datetime(datetime(2020, 11, 1, 0, 0))

survey_df1 = survey_df[survey_df['End datetime'] <= daylight].copy()
survey_df1['Start datetime'] = survey_df1['Start datetime'].apply(lambda x: x + timedelta(hours=5))
survey_df1['End datetime'] = survey_df1['End datetime'].apply(lambda x: x + timedelta(hours=5))

survey_df2 = survey_df.loc[survey_df['End datetime'] > daylight].copy()
survey_df2['Start datetime'] = survey_df2['Start datetime'].apply(lambda x: x + timedelta(hours=6))
survey_df2['End datetime'] = survey_df2['End datetime'].apply(lambda x: x + timedelta(hours=6))

survey_df = pd.concat([survey_df1, survey_df2], ignore_index=True)
# survey_df = survey_df.loc[survey_df['Stress level'] != 1.0]

survey_df.reset_index(drop=True, inplace=True)

# Label Data
print('Labelling ...')
ids = df['id'].unique()
results=[]
for i in range(len(ids)):
    id=ids[i]
    new_df = pd.DataFrame(columns=['X', 'Y', 'Z', 'EDA', 'HR', 'TEMP', 'id', 'datetime', 'label'])

    sdf = df[df['id'] == id].copy()
    survey_sdf = survey_df[survey_df['ID'] == id].copy()

    for _, survey_row in survey_sdf.iterrows():
        ssdf = sdf[(sdf['datetime'] >= survey_row['Start datetime']) & (sdf['datetime'] <= survey_row['End datetime'])].copy()

        if not ssdf.empty:
            ssdf['label'] = np.repeat(survey_row['Stress level'], len(ssdf.index))
            new_df = pd.concat([new_df, ssdf], ignore_index=True)
        else:
            print(f"{survey_row['ID']} is missing label {survey_row['Stress level']} at {survey_row['Start datetime']} to {survey_row['End datetime']}")
        
    results.append(new_df)




new_df = pd.concat(results, ignore_index=True)

print('Saving ...')
new_df.to_csv(os.path.join(PATH, 'merged_data_labeled.csv'), index=False)
print('Done')

Reading 1 ...
Reading 2 ...
Converting ...
Labelling ...
5C is missing label 1.0 at 2020-04-15 13:00:00 to 2020-04-15 14:00:00
5C is missing label 0.0 at 2020-06-12 07:00:00 to 2020-06-12 08:00:00
6D is missing label 1.0 at 2020-06-03 07:00:00 to 2020-06-03 09:00:00
7A is missing label 2.0 at 2020-07-07 19:16:00 to 2020-07-07 19:27:00
7A is missing label 2.0 at 2020-07-07 19:50:00 to 2020-07-07 20:09:00
7A is missing label 0.0 at 2020-07-07 20:24:00 to 2020-07-07 20:57:00
Saving ...
Done
